# Method 2: 2D Convolutional Neural Network
## GTZAN Music Genre Classification

This notebook implements 2D CNN for spectral pattern recognition using Mel-spectrograms.

**Expected Accuracy:** 88-92%

## 1. Import Libraries

In [1]:
import numpy as np
import librosa
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Conv2D, BatchNormalization, MaxPooling2D,
                                     GlobalAveragePooling2D, Dense, Dropout)
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm

# Set style for plots
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# Set random seeds for reproducibility
np.random.seed(42)
import tensorflow as tf
tf.random.set_seed(42)

## 2. Feature Extraction - Mel-Spectrogram

In [2]:
def extract_melspectrogram(audio_path, sr=22050, duration=30.0, n_mels=64):
    """
    Extract mel-spectrogram features
    Returns: (time_frames, n_mels) array
    """
    # Load audio
    audio, sr = librosa.load(audio_path, sr=sr, mono=True, duration=duration)
    
    # Compute mel-spectrogram
    melspec = librosa.feature.melspectrogram(
        y=audio, sr=sr, n_mels=n_mels,
        n_fft=2048, hop_length=512, power=2.0
    )
    
    # Convert to log scale (dB)
    melspec_db = librosa.power_to_db(melspec, ref=np.max)
    
    # Transpose to (time, frequency)
    return melspec_db.T

## 3. Visualize Sample Mel-Spectrogram

In [3]:
# Visualize a sample mel-spectrogram (update path to a sample audio file)
# sample_path = 'path/to/sample.wav'
# sample_melspec = extract_melspectrogram(sample_path)

# plt.figure(figsize=(14, 6))
# librosa.display.specshow(sample_melspec.T, sr=22050, hop_length=512, 
#                          x_axis='time', y_axis='mel', cmap='viridis')
# plt.colorbar(format='%+2.0f dB')
# plt.title('Sample Mel-Spectrogram', fontsize=14, fontweight='bold')
# plt.tight_layout()
# plt.show()

## 4. Load GTZAN Dataset

In [4]:
def load_gtzan_melspec(data_path, target_length=1291):
    """
    Load GTZAN dataset and extract mel-spectrogram features
    """
    genres = ['blues', 'classical', 'country', 'disco', 'hiphop', 
              'jazz', 'metal', 'pop', 'reggae', 'rock']
    
    features = []
    labels = []
    
    print("Loading GTZAN dataset and extracting Mel-spectrograms...")
    for genre_idx, genre in enumerate(genres):
        print(f"Processing {genre}...")
        genre_path = os.path.join(data_path, genre)
        
        files = [f for f in os.listdir(genre_path) if f.endswith('.wav')]
        
        for filename in tqdm(files, desc=f"{genre}"):
            if filename == 'jazz.00054.wav':
                print(f"Skipping corrupted file: {filename}")
                continue
            filepath = os.path.join(genre_path, filename)
            try:
                melspec = extract_melspectrogram(filepath)
                
                # Pad or truncate to target length
                if melspec.shape[0] < target_length:
                    pad_width = target_length - melspec.shape[0]
                    melspec = np.pad(melspec, ((0, pad_width), (0, 0)), 
                                   mode='constant')
                else:
                    melspec = melspec[:target_length, :]
                
                features.append(melspec)
                labels.append(genre_idx)
            except Exception as e:
                print(f"Error processing {filepath}: {e}")
    
    return np.array(features), np.array(labels), genres

In [5]:
# UPDATE THIS PATH to your GTZAN dataset location
data_path = 'path/to/gtzan/genres'

# Load data
features, labels, genre_names = load_gtzan_melspec('/Users/narac0503/GIT/GTZAN Dataset Classification/GTZAN-Dataset-Classification/gtzan-classification/data/gtzan/genres_original')

print(f"\nDataset loaded successfully!")
print(f"Features shape: {features.shape}")
print(f"Labels shape: {labels.shape}")
print(f"Genres: {genre_names}")

Loading GTZAN dataset and extracting Mel-spectrograms...
Processing blues...


blues:   0%|          | 0/100 [00:00<?, ?it/s]/opt/anaconda3/envs/gtzan/lib/python3.9/site-packages/librosa/core/intervals.py:8: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename
blues: 100%|██████████| 100/100 [00:02<00:00, 40.85it/s]


Processing classical...


classical: 100%|██████████| 100/100 [00:01<00:00, 55.55it/s]


Processing country...


...

Processing reggae...


reggae: 100%|██████████| 100/100 [00:01<00:00, 51.67it/s]


Processing rock...


rock: 100%|██████████| 100/100 [00:01<00:00, 55.42it/s]



Dataset loaded successfully!
Features shape: (999, 1291, 64)
Labels shape: (999,)
Genres: ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']


## 5. Data Preparation

In [6]:
# Convert labels to one-hot encoding
labels_onehot = to_categorical(labels, num_classes=10)

# Split: 80% train, 10% validation, 10% test
X_temp, X_test, y_temp, y_test = train_test_split(
    features, labels_onehot, 
    test_size=0.1, stratify=labels, random_state=42
)

X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp,
    test_size=0.111, stratify=np.argmax(y_temp, axis=1), random_state=42
)

print(f"Training set: {X_train.shape}")
print(f"Validation set: {X_val.shape}")
print(f"Test set: {X_test.shape}")

Training set: (799, 1291, 64)
Validation set: (100, 1291, 64)
Test set: (100, 1291, 64)


In [7]:
# Normalize features (Z-score normalization)
scaler = StandardScaler()
X_train_norm = scaler.fit_transform(
    X_train.reshape(-1, X_train.shape[-1])
).reshape(X_train.shape)

X_val_norm = scaler.transform(
    X_val.reshape(-1, X_val.shape[-1])
).reshape(X_val.shape)

X_test_norm = scaler.transform(
    X_test.reshape(-1, X_test.shape[-1])
).reshape(X_test.shape)

# Add channel dimension for CNN
X_train_norm = X_train_norm[..., np.newaxis]
X_val_norm = X_val_norm[..., np.newaxis]
X_test_norm = X_test_norm[..., np.newaxis]

print(f"\nNormalized data shapes:")
print(f"Train: {X_train_norm.shape}")
print(f"Val: {X_val_norm.shape}")
print(f"Test: {X_test_norm.shape}")


Normalized data shapes:
Train: (799, 1291, 64, 1)
Val: (100, 1291, 64, 1)
Test: (100, 1291, 64, 1)


## 6. Build 2D CNN Model

In [8]:
from tensorflow.keras.layers import LeakyReLU
def build_cnn_model(input_shape):
    """
    Build Improved 2D CNN model
    """
    model = Sequential([
        # Conv Block 1
        Conv2D(256, (3, 3), padding='same', input_shape=input_shape, name='conv2d_1'),
        LeakyReLU(alpha=0.1),
        BatchNormalization(name='bn_1'),
        MaxPooling2D((2, 2), name='maxpool_1'),
        
        # Conv Block 2
        Conv2D(64, (3, 3), padding='same', name='conv2d_2'),
        LeakyReLU(alpha=0.1),
        BatchNormalization(name='bn_2'),
        MaxPooling2D((2, 2), name='maxpool_2'),
        
        # Conv Block 3
        Conv2D(128, (3, 3), padding='same', name='conv2d_3'),
        LeakyReLU(alpha=0.1),
        BatchNormalization(name='bn_3'),
        MaxPooling2D((2, 2), name='maxpool_3'),
        
        # Conv Block 4
        Conv2D(256, (3, 3), padding='same', name='conv2d_4'),
        LeakyReLU(alpha=0.1),
        BatchNormalization(name='bn_4'),
        GlobalAveragePooling2D(name='global_avgpool'),
        
        # Classification head
        Dense(256, name='dense_1'),
        LeakyReLU(alpha=0.1),
        Dropout(0.4, name='dropout'),
        Dense(10, activation='softmax', name='output')
    ])
    
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Build model
input_shape = (X_train_norm.shape[1], X_train_norm.shape[2], 1)
model = build_cnn_model(input_shape)

# Display model architecture
model.summary()

2025-12-07 15:40:08.816050: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4 Pro
2025-12-07 15:40:08.816086: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 24.00 GB
2025-12-07 15:40:08.816100: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 8.88 GB
2025-12-07 15:40:08.816136: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-12-07 15:40:08.816148: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 1291, 64, 256)     2560      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 1291, 64, 256)     0         
                                                                 
 bn_1 (BatchNormalization)   (None, 1291, 64, 256)     1024      
                                                                 
 maxpool_1 (MaxPooling2D)    (None, 645, 32, 256)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 645, 32, 64)       147520    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 645, 32, 64)       0         
                                                                 
 bn_2 (BatchNormalization)   (None, 645, 32, 64)       2

## 7. Train Model

In [9]:
# Train model
history = model.fit(
    X_train_norm, y_train,
    validation_data=(X_val_norm, y_val),
    batch_size=32,
    epochs=100,
    verbose=1
)

Epoch 1/100


2025-12-07 15:40:09.500612: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


25/25 [==============================] - 17s 596ms/step - loss: 1.8407 - accuracy: 0.3554 - val_loss: 2.1433 - val_accuracy: 0.3100
Epoch 2/100
25/25 [==============================] - 14s 564ms/step - loss: 1.4670 - accuracy: 0.4831 - val_loss: 2.5392 - val_accuracy: 0.1700
Epoch 3/100
25/25 [==============================] - 14s 556ms/step - loss: 1.2281 - accuracy: 0.5807 - val_loss: 2.8736 - val_accuracy: 0.1000
Epoch 4/100
25/25 [==============================] - 14s 565ms/step - loss: 1.1103 - accuracy: 0.6133 - val_loss: 3.1714 - val_accuracy: 0.1400
Epoch 5/100
25/25 [==============================] - 14s 558ms/step - loss: 1.0124 - accuracy: 0.6546 - val_loss: 3.9482 - val_accuracy: 0.1000
Epoch 6/100
25/25 [==============================] - 14s 566ms/step - loss: 0.9676 - accuracy: 0.6683 - val_loss: 4.3603 - val_accuracy: 0.1000
Epoch 7/100
25/25 [==============================] - 15s 587ms/step - loss: 0.8458 - accuracy: 0.6959 - val_loss: 3.9668 - val_accuracy: 0.1600
Epoc

## 8. Plot Training History

In [10]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Loss plot
ax1.plot(history.history['loss'], label='Train Loss', linewidth=2)
ax1.plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Loss', fontsize=12)
ax1.set_title('2D CNN - Training and Validation Loss', fontsize=14, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(alpha=0.3)

# Accuracy plot
ax2.plot(history.history['accuracy'], label='Train Accuracy', linewidth=2)
ax2.plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Accuracy', fontsize=12)
ax2.set_title('2D CNN - Training and Validation Accuracy', fontsize=14, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('cnn_training_history.png', dpi=300, bbox_inches='tight')
plt.show()

## 9. Evaluate Model

In [11]:
# Predictions
y_pred = model.predict(X_test_norm)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(y_test, axis=1)

# Calculate accuracy
accuracy = np.mean(y_pred_labels == y_true_labels)
print(f"Test Accuracy: {accuracy*100:.2f}%")

4/4 [==============================] - 0s 102ms/step
Test Accuracy: 84.00%


In [12]:
# Classification report
print("\n" + "="*60)
print("2D CNN - Classification Report")
print("="*60)
print(classification_report(y_true_labels, y_pred_labels, target_names=genre_names))


2D CNN - Classification Report
              precision    recall  f1-score   support

       blues       0.77      1.00      0.87        10
   classical       0.90      0.90      0.90        10
     country       0.88      0.70      0.78        10
       disco       0.75      0.90      0.82        10
      hiphop       0.82      0.90      0.86        10
        jazz       0.89      0.80      0.84        10
       metal       1.00      1.00      1.00        10
         pop       0.80      0.80      0.80        10
      reggae       0.89      0.80      0.84        10
        rock       0.75      0.60      0.67        10

    accuracy                           0.84       100
   macro avg       0.84      0.84      0.84       100
weighted avg       0.84      0.84      0.84       100



## 10. Confusion Matrix

In [13]:
# Confusion matrix
cm = confusion_matrix(y_true_labels, y_pred_labels)

plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=genre_names, yticklabels=genre_names,
            cbar_kws={'label': 'Count'})
plt.xlabel('Predicted Genre', fontsize=12, fontweight='bold')
plt.ylabel('True Genre', fontsize=12, fontweight='bold')
plt.title('2D CNN - Confusion Matrix', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig('cnn_confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

## 11. Per-Genre Performance

In [14]:
from sklearn.metrics import precision_recall_fscore_support

# Calculate per-genre metrics
precision, recall, f1, support = precision_recall_fscore_support(
    y_true_labels, y_pred_labels, average=None
)

# Create bar plot
fig, ax = plt.subplots(figsize=(14, 6))
x = np.arange(len(genre_names))
width = 0.25

ax.bar(x - width, precision, width, label='Precision', alpha=0.8)
ax.bar(x, recall, width, label='Recall', alpha=0.8)
ax.bar(x + width, f1, width, label='F1-Score', alpha=0.8)

ax.set_xlabel('Genre', fontsize=12, fontweight='bold')
ax.set_ylabel('Score', fontsize=12, fontweight='bold')
ax.set_title('2D CNN - Per-Genre Performance Metrics', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(genre_names, rotation=45, ha='right')
ax.legend(fontsize=10)
ax.grid(axis='y', alpha=0.3)
ax.set_ylim(0, 1.0)

plt.tight_layout()
plt.savefig('cnn_per_genre_metrics.png', dpi=300, bbox_inches='tight')
plt.show()

## 12. Save Model

In [16]:
# Save model
model.save('cnn_model.keras')
print("✓ Model saved: cnn_model.keras")

# Save training history
np.save('cnn_history.npy', history.history)
print("✓ Training history saved: cnn_history.npy")

✓ Model saved: cnn_model.keras
✓ Training history saved: cnn_history.npy


## Summary

This notebook implemented a 2D CNN model for GTZAN music genre classification:

- **Input Features**: Mel-spectrogram (1291 × 64 time-frequency representation)
- **Architecture**: 3 Conv2D blocks (32→64→128 filters) with batch normalization
- **Training**: Early stopping with patience=10
- **Expected Accuracy**: 88-92%

The model successfully captures spectral patterns in music for accurate genre classification.